In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Celiac_Disease"
cohort = "GSE112102"

# Input paths
in_trait_dir = "../../input/GEO/Celiac_Disease"
in_cohort_dir = "../../input/GEO/Celiac_Disease/GSE112102"

# Output paths
out_data_file = "../../output/preprocess/Celiac_Disease/GSE112102.csv"
out_gene_data_file = "../../output/preprocess/Celiac_Disease/gene_data/GSE112102.csv"
out_clinical_data_file = "../../output/preprocess/Celiac_Disease/clinical_data/GSE112102.csv"
json_path = "../../output/preprocess/Celiac_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Analyze gene expression data availability
is_gene_available = True  # Based on the Series_summary, this dataset contains gene expression data

# 2. Analyze variable availability and conversion functions
# 2.1 Identify rows for trait, age, and gender
trait_row = 1  # The trait information is in row 1 (group: CeD, control, FDR)
age_row = 2    # Age information is in row 2
gender_row = 4  # Gender information is in row 4

# 2.2 Define conversion functions
def convert_trait(value):
    """Convert trait values to binary (1 for CeD, 0 for control, None for FDR)"""
    if not isinstance(value, str):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() == 'ced':
        return 1  # Celiac Disease
    elif value.lower() == 'control':
        return 0  # Control
    else:
        return None  # FDR (First Degree Relatives) are neither cases nor controls

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if not isinstance(value, str):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for Female, 1 for Male)"""
    if not isinstance(value, str):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save metadata for initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Get the clinical data by filtering and processing
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of extracted clinical data:")
    print(preview_df(clinical_df))
    
    # Save clinical data to CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# These identifiers begin with "ILMN_" which indicates they are Illumina probe IDs,
# specifically from an Illumina BeadArray microarray platform.
# These are not human gene symbols but rather probe identifiers that need to be
# mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns for mapping
# From the gene annotation preview, we can see that 'ID' contains the probe identifiers (ILMN_*) 
# and 'Symbol' contains the gene symbols
prob_col = 'ID'  # Column with probe IDs matching the gene expression data index
gene_col = 'Symbol'  # Column with gene symbols

# 2. Create the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping - shape: {gene_data.shape}")
print("First 5 genes in the mapped data:")
print(gene_data.index[:5])

# Save the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was saved in Step 2
clinical_data = pd.read_csv(out_clinical_data_file)

# Convert to proper format for linking
clinical_df = pd.DataFrame()
sample_ids = clinical_data.columns

# Create proper clinical dataframe with samples as columns and features as rows
clinical_df[trait] = clinical_data.iloc[0]
clinical_df['Age'] = clinical_data.iloc[1]
clinical_df['Gender'] = clinical_data.iloc[2]
clinical_df.index = [trait, 'Age', 'Gender']
clinical_df.columns = sample_ids

print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(clinical_df))

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# Transpose linked data to have samples as rows and features as columns
linked_data = linked_data.T

print("After transposing:")
print(f"Linked data shape: {linked_data.shape}")
print("Actual column names in linked_data:", linked_data.columns.tolist()[:10])  # Show first 10 columns

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from Celiac Disease patients and controls."
)

# 7. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data from Step 2
clinical_data_path = out_clinical_data_file
clinical_data = pd.read_csv(clinical_data_path)
print(f"Loaded clinical data from {clinical_data_path}")

# Create a proper clinical DataFrame with rows as features and columns as samples
# First, get column names from the CSV (these are the sample IDs)
sample_ids = clinical_data.columns.tolist()
# Extract and prepare the clinical features
trait_values = clinical_data.iloc[0].values
age_values = clinical_data.iloc[1].values
gender_values = clinical_data.iloc[2].values

# Create DataFrame with correct format for linking
clinical_df = pd.DataFrame({
    trait: trait_values,
    'Age': age_values,
    'Gender': gender_values
}, index=sample_ids).T

print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.iloc[:, :5])  # Show first 5 columns

# 3. Link clinical and genetic data
linked_data = pd.concat([clinical_df, gene_data], axis=0)
print(f"Linked data shape after concatenation: {linked_data.shape}")

# Transpose to have samples as rows and features as columns
linked_data = linked_data.T
print(f"Linked data shape after transpose: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from Celiac Disease patients and controls. FDR samples were excluded from the trait analysis."
)

# 7. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")